In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_full = pd.read_csv("LA-142DPI-metadata-feature-table.csv")
data_a = data_full[data_full["Treatmentgroup"].isin(["Group1C","Group3E"])]

data_restore = pd.read_csv("LA-mass-restored-15.csv")
data_restore_list = data_restore.filename.to_list()
data_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 2:9]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api
dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(df_pval_list, method='indep')
df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list, dr_fdr_pval[0], dr_fdr_pval[1])), columns=['col_name','corr', 'corr_pval', 'FDR_rejected', 'FDR_pval'])

df_final.to_csv('LA-142DPI-group15-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.Weight..Body.Weight, Liver.Weight..Body.Weight, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X258.111_0.302, X608.393_3.098, X616.308_2.766]
Index: []
['Heart.Weight..Body.Weight-X258.111_0.302', 'Heart.Weight..Body.Weight-X608.393_3.098', 'Heart.Weight..Body.Weight-X616.308_2.766', 'Liver.Weight..Body.Weight-X258.111_0.302', 'Liver.Weight..Body.Weight-X608.393_3.098', 'Liver.Weight..Body.Weight-X616.308_2.766', 'Ejection.Fraction-X258.111_0.302', 'Ejection.Fraction-X608.393_3.098', 'Ejection.Fraction-X616.308_2.766', 'PR.Interval..s.-X258.111_0.302', 'PR.Interval..s.-X608.393_3.098', 'PR.Interval..s.-X616.308_2.766', 'QTc..s.-X258.111_0.302', 'QTc..s.-X608.393_3.098', 'QTc..s.-X616.308_2.766', 'P.Amplitude..V.-X258.111_0.302', 'P.Amplitude..V.-X608.393_3.098', 'P.Amplitude..V.-X616.308_2.766', 'CD3.CD8.IFNg.-X258.111_0.302', 'CD3.CD8.IFNg.-X608.393_3.098', 'CD3.CD8.IFNg.-X616.308_2.766'